# chunk

> internal utilities for chunkwise data processing

In [ ]:
#| default_exp chunk_

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import itertools
from moraine.rtree import HilbertRtree
from scipy.spatial import KDTree
from joblib import Parallel, delayed
from numba import njit

In [ ]:
#| export
def fill_slice(
    shape, # numpy arr, cupy arr, zarr,
    slices, # tuple of slice object, len == data_arr.ndim
):
    out_slices = []
    for i in range(len(slices)):
        slice_i = slices[i]
        if slice_i.start is None:
            start = 0
        else:
            start = slice_i.start
        assert start>=0
        assert start<shape[i]
        if slice_i.stop is None:
            stop = shape[i]
        else:
            stop = slice_i.stop
        assert stop > start
        assert stop <= shape[i]
        assert (slice_i.step is None) or (slice_i.step == 1)
        step = 1
        out_slices.append(slice(start,stop,step))
    return tuple(out_slices)

In [ ]:
#| export
def all_chunk_slices(
    shape, # np.array, cp.array,zarr
    chunks,
):
    '''get the slices for every input chunks'''
    out_slices = []
    for shape_, chunk_ in zip(shape,chunks):
        if chunk_ <0: chunk_ = shape_
        bound_1dim = np.arange(0,shape_+chunk_,chunk_)
        if bound_1dim[-1] > shape_: bound_1dim[-1] = shape_

        slice_1dim = []
        for j in range(bound_1dim.shape[0]-1):
            slice_1dim.append(slice(int(bound_1dim[j]),int(bound_1dim[j+1])))
        out_slices.append(slice_1dim)
    out_slices = list(itertools.product(*out_slices))
    return out_slices

usage:

In [ ]:
a = np.zeros((50,100,11))
chunks = (15,40,6)
all_chunk_slices(a.shape,chunks)

[(slice(0, 15, None), slice(0, 40, None), slice(0, 6, None)),
 (slice(0, 15, None), slice(0, 40, None), slice(6, 11, None)),
 (slice(0, 15, None), slice(40, 80, None), slice(0, 6, None)),
 (slice(0, 15, None), slice(40, 80, None), slice(6, 11, None)),
 (slice(0, 15, None), slice(80, 100, None), slice(0, 6, None)),
 (slice(0, 15, None), slice(80, 100, None), slice(6, 11, None)),
 (slice(15, 30, None), slice(0, 40, None), slice(0, 6, None)),
 (slice(15, 30, None), slice(0, 40, None), slice(6, 11, None)),
 (slice(15, 30, None), slice(40, 80, None), slice(0, 6, None)),
 (slice(15, 30, None), slice(40, 80, None), slice(6, 11, None)),
 (slice(15, 30, None), slice(80, 100, None), slice(0, 6, None)),
 (slice(15, 30, None), slice(80, 100, None), slice(6, 11, None)),
 (slice(30, 45, None), slice(0, 40, None), slice(0, 6, None)),
 (slice(30, 45, None), slice(0, 40, None), slice(6, 11, None)),
 (slice(30, 45, None), slice(40, 80, None), slice(0, 6, None)),
 (slice(30, 45, None), slice(40, 80, None

In [ ]:
a = np.zeros(343)
all_chunk_slices(a.shape,(15,))

[(slice(0, 15, None),),
 (slice(15, 30, None),),
 (slice(30, 45, None),),
 (slice(45, 60, None),),
 (slice(60, 75, None),),
 (slice(75, 90, None),),
 (slice(90, 105, None),),
 (slice(105, 120, None),),
 (slice(120, 135, None),),
 (slice(135, 150, None),),
 (slice(150, 165, None),),
 (slice(165, 180, None),),
 (slice(180, 195, None),),
 (slice(195, 210, None),),
 (slice(210, 225, None),),
 (slice(225, 240, None),),
 (slice(240, 255, None),),
 (slice(255, 270, None),),
 (slice(270, 285, None),),
 (slice(285, 300, None),),
 (slice(300, 315, None),),
 (slice(315, 330, None),),
 (slice(330, 343, None),)]

In [ ]:
#| export
def all_chunk_slices_with_overlap(
    shape, 
    chunks, 
    depths,
):
    '''get the slices for every input chunks with overlap'''
    out_slices = []
    for shape_, chunk_, depth_ in zip(shape,chunks,depths):
        if chunk_ <0: chunk_ = shape_
        starts_1dim = np.arange(-depth_,shape_-depth_,chunk_)
        starts_1dim[starts_1dim<0] = 0
        ends_1dim = np.arange(chunk_+depth_,shape_+chunk_+depth_,chunk_)
        ends_1dim[ends_1dim>shape_] = shape_

        slice_1dim = []
        for j in range(starts_1dim.shape[0]):
            slice_1dim.append(slice(int(starts_1dim[j]),int(ends_1dim[j])))
        out_slices.append(slice_1dim)
    out_slices = list(itertools.product(*out_slices))
    return out_slices

In [ ]:
a = np.zeros((50,100,11))
chunks = (15,40,6)
depth = (1,2,0)
all_chunk_slices_with_overlap(a.shape,chunks,depth)

[(slice(0, 16, None), slice(0, 42, None), slice(0, 6, None)),
 (slice(0, 16, None), slice(0, 42, None), slice(6, 11, None)),
 (slice(0, 16, None), slice(38, 82, None), slice(0, 6, None)),
 (slice(0, 16, None), slice(38, 82, None), slice(6, 11, None)),
 (slice(0, 16, None), slice(78, 100, None), slice(0, 6, None)),
 (slice(0, 16, None), slice(78, 100, None), slice(6, 11, None)),
 (slice(14, 31, None), slice(0, 42, None), slice(0, 6, None)),
 (slice(14, 31, None), slice(0, 42, None), slice(6, 11, None)),
 (slice(14, 31, None), slice(38, 82, None), slice(0, 6, None)),
 (slice(14, 31, None), slice(38, 82, None), slice(6, 11, None)),
 (slice(14, 31, None), slice(78, 100, None), slice(0, 6, None)),
 (slice(14, 31, None), slice(78, 100, None), slice(6, 11, None)),
 (slice(29, 46, None), slice(0, 42, None), slice(0, 6, None)),
 (slice(29, 46, None), slice(0, 42, None), slice(6, 11, None)),
 (slice(29, 46, None), slice(38, 82, None), slice(0, 6, None)),
 (slice(29, 46, None), slice(38, 82, None

In [ ]:
#| export
def chunkwise_slicing_mapping(
    shape,
    chunks,
    depths,
):
    '''get the slices for every input chunks with overlap
    output chunks without overlap and their mapping slices'''
    in_slices = []; out_slices = []; map_slices = []
    for shape_, chunk_, depth_ in zip(shape,chunks,depths):
        if chunk_ <0: chunk_ = shape_
        in_starts_1dim = np.arange(-depth_,shape_-depth_,chunk_)
        in_starts_1dim[in_starts_1dim<0] = 0
        in_ends_1dim = np.arange(chunk_+depth_,shape_+chunk_+depth_,chunk_)
        in_ends_1dim[in_ends_1dim>shape_] = shape_
        out_starts_1dim = np.arange(0,shape_,chunk_)
        out_ends_1dim = np.arange(chunk_,shape_+chunk_,chunk_)
        out_ends_1dim[out_ends_1dim>shape_] = shape_

        assert len(in_starts_1dim) == len(out_starts_1dim)
        assert len(in_ends_1dim) == len(out_ends_1dim)

        in_slice_1dim = []; out_slice_1dim = []; map_slice_1dim = []
        for in_start, in_end, out_start, out_end in zip(in_starts_1dim, in_ends_1dim, out_starts_1dim, out_ends_1dim):
            in_slice_1dim.append(slice(int(in_start),int(in_end)))
            out_slice_1dim.append(slice(int(out_start),int(out_end)))
            offset = out_start-in_start
            map_slice_1dim.append(slice(int(offset),int(offset+out_end-out_start)))

        in_slices.append(in_slice_1dim)
        out_slices.append(out_slice_1dim)
        map_slices.append(map_slice_1dim)

    in_slices = list(itertools.product(*in_slices))
    out_slices = list(itertools.product(*out_slices))
    map_slices = list(itertools.product(*map_slices))
    return in_slices, out_slices, map_slices

In [ ]:
a = np.zeros((50,100))
chunks = (15,40)
depth = [1,2] # or {0:1, 1:2, 2:3}
chunkwise_slicing_mapping(a.shape,chunks,depth)

([(slice(0, 16, None), slice(0, 42, None)),
  (slice(0, 16, None), slice(38, 82, None)),
  (slice(0, 16, None), slice(78, 100, None)),
  (slice(14, 31, None), slice(0, 42, None)),
  (slice(14, 31, None), slice(38, 82, None)),
  (slice(14, 31, None), slice(78, 100, None)),
  (slice(29, 46, None), slice(0, 42, None)),
  (slice(29, 46, None), slice(38, 82, None)),
  (slice(29, 46, None), slice(78, 100, None)),
  (slice(44, 50, None), slice(0, 42, None)),
  (slice(44, 50, None), slice(38, 82, None)),
  (slice(44, 50, None), slice(78, 100, None))],
 [(slice(0, 15, None), slice(0, 40, None)),
  (slice(0, 15, None), slice(40, 80, None)),
  (slice(0, 15, None), slice(80, 100, None)),
  (slice(15, 30, None), slice(0, 40, None)),
  (slice(15, 30, None), slice(40, 80, None)),
  (slice(15, 30, None), slice(80, 100, None)),
  (slice(30, 45, None), slice(0, 40, None)),
  (slice(30, 45, None), slice(40, 80, None)),
  (slice(30, 45, None), slice(80, 100, None)),
  (slice(45, 50, None), slice(0, 40, No

In [ ]:
#| export
@njit(fastmath=True, cache=True, nogil=True)
def normalize_xy_stack(x_chunk, y_chunk):
    """
    Normalize x, y to [0, 1] range and stack into shape (N, 2).
    Returns normalized positions and (x_min, x_max, y_min, y_max)
    """
    x_min = x_chunk.min()
    x_max = x_chunk.max()
    y_min = y_chunk.min()
    y_max = y_chunk.max()

    # Avoid division by zero
    x_range = x_max - x_min
    y_range = y_max - y_min
    if x_range == 0.0:
        x_range = 1.0
    if y_range == 0.0:
        y_range = 1.0

    n = x_chunk.size
    pos_norm = np.empty((n, 2), dtype=x_chunk.dtype)
    for i in range(n):
        pos_norm[i, 0] = (x_chunk[i] - x_min) / x_range
        pos_norm[i, 1] = (y_chunk[i] - y_min) / y_range

    return pos_norm, x_min, x_max, y_min, y_max

In [ ]:
#| export
@njit(fastmath=True, cache=True, nogil=True)
def get_boundary_idx(x_norm, y_norm, k=128, bound_ratio_init=0.05, bound_ratio_step=0.05):
    """
    Select boundary point indices from normalized coordinates.

    Args:
        x_norm, y_norm: normalized 1D arrays (0–1 range)
        k: least number of boundary points
        bound_ratio_init: initial boundary width ratio
        bound_ratio_step: increment step if not enough points

    Returns:
        bound_idx: 1D array of indices of boundary points
    """
    n_chunk = x_norm.size
    if n_chunk < 2 * k:
        return np.arange(n_chunk)

    br = bound_ratio_init
    bound_idx = np.empty(n_chunk, dtype=np.int64)
    while True:
        count = 0
        for i in range(n_chunk):
            x = x_norm[i]
            y = y_norm[i]
            if (x < br) or (x > 1.0 - br) or (y < br) or (y > 1.0 - br):
                bound_idx[count] = i
                count += 1

        if count >= k or br >= 0.5:
            return bound_idx[:count]

        br += bound_ratio_step

In [ ]:
#| export
@njit(fastmath=True, cache=True, nogil=True)
def normalize_and_get_boundary(x_chunk, y_chunk, k=128, bound_ratio_init=0.05, bound_ratio_step=0.05):
    """
    Combined normalization and boundary index selection.
    """
    pos_norm, x_min, x_max, y_min, y_max = normalize_xy_stack(x_chunk, y_chunk)
    bound_idx = get_boundary_idx(pos_norm[:, 0], pos_norm[:, 1], k, bound_ratio_init, bound_ratio_step)
    return pos_norm, x_min, x_max, y_min, y_max, bound_idx

In [ ]:
#| export
def process_chunk(i, bound, x, y, k, chunks, rtree_bounds_tree, bound_ratio_init, bound_ratio_step):
    start = int(bound[i])
    end = int(bound[i + 1])
    x_chunk = x[start:end]
    y_chunk = y[start:end]
    n_chunk = end - start

    # === Step 1. Normalize + boundary detection (Numba) ===
    pos_norm, x_min, x_max, y_min, y_max, bound_idx = normalize_and_get_boundary(
        x_chunk, y_chunk, k, bound_ratio_init, bound_ratio_step
    )
    x_norm, y_norm = pos_norm[:, 0], pos_norm[:, 1]

    # === Step 2. Estimate distance of k-th NN for boundary points ===
    tree = KDTree(pos_norm)
    dd = tree.query(pos_norm[bound_idx], k=[k], workers=1)[0]
    max_dd = np.max(dd)

    # === Step 3. Compute halo region and query ===
    y_halo_size = max_dd * (y_max - y_min)
    x_halo_size = max_dd * (x_max - x_min)
    halo_bounds = [x_min - x_halo_size, y_min - y_halo_size,
                   x_max + x_halo_size, y_max + y_halo_size]
    rtree = HilbertRtree(rtree_bounds_tree, x.shape[0], chunks)
    halo_idx = rtree.bbox_query(halo_bounds, x, y)

    # === Step 4. Build KDTree for halo points and find input indices ===
    x_halo, y_halo = x[halo_idx], y[halo_idx]
    tree = KDTree(np.stack((x_halo, y_halo), axis=-1))
    in_idx = tree.query(np.stack((x_chunk, y_chunk), axis=-1), k=k, workers=1)[1]
    in_idx = np.unique(in_idx)
    in_idx = halo_idx[in_idx]  # map to global indices

    # === Step 5. Compute mapping indices (local to chunk)
    map_mask = (in_idx >= start) & (in_idx < end)
    map_idx = np.flatnonzero(map_mask)

    return in_idx, slice(start, end), map_idx

In [ ]:
#| export
def chunkwise_knn_mapping(x, y, chunks, k=128, n_jobs=-1, bound_ratio_init=0.1, bound_ratio_step=0.1):
    """
    Compute KNN mapping chunkwise with halo expansion.

    Returns
    -------
    in_indices : list of np.ndarray
        Indices of halo points per chunk.
    out_slices : list of slice
        Chunk output slices.
    map_indices : list of np.ndarray
        Mapping indices inside halo arrays.
    """
    n = y.shape[0]
    bound = np.arange(0, n + chunks, chunks)
    if bound[-1] > n:
        bound[-1] = n

    # Build spatial index
    rtree = HilbertRtree.build(x, y, page_size=chunks)
    rtree_bounds_tree = rtree._bounds_tree

    results = Parallel(n_jobs=n_jobs,backend='loky')(
        delayed(process_chunk)(
            i, bound, x, y, k, chunks, rtree_bounds_tree, bound_ratio_init, bound_ratio_step
        )
        for i in range(bound.shape[0] - 1)
    )

    in_indices, out_slices, map_indices = zip(*results)
    return list(in_indices), list(out_slices), list(map_indices)

In [ ]:
import zarr

In [ ]:
# lon = zarr.open('../Tutorials/CLI/ps_processing/hix/ps_can_lon.zarr/',mode='r')[:]
# lat = zarr.open('../Tutorials/CLI/ps_processing/hix/ps_can_lat.zarr/',mode='r')[:]

lon = zarr.open('../../../William_Sound/192021/ps_processing/hix/ps_can_lon.zarr/',mode='r')[:]
lat = zarr.open('../../../William_Sound/192021/ps_processing/hix/ps_can_lat.zarr/',mode='r')[:]

In [ ]:
%%time
in_indices, out_slices, map_indices = chunkwise_knn_mapping(lat, lon, 40000)

CPU times: user 3.68 s, sys: 796 ms, total: 4.47 s
Wall time: 15.7 s


In [ ]:
#| hide
def chunkwise_knn_mapping_(
    x,
    y,
    chunks:int,
    k = 128
):
    n = y.shape[0]
    bound = np.arange(0,n+chunks,chunks)
    if bound[-1] > n: bound[-1] = n
    rtree = HilbertRtree.build(x,y,page_size=chunks)

    out_slices = []
    in_indices = []
    map_indices = []
    for i in range(bound.shape[0]-1):
        start = int(bound[i])
        end = int(bound[i+1])
        out_slices.append(slice(start,end))

        y_, x_ = y[start:end], x[start:end]
        # normalize x and y here should be better and save the ratio!
        y_max, y_min, x_max, x_min = y_.max(), y_.min(), x_.max(), x_.min()
        y_norm = (y_ - y_min)/(y_max - y_min)
        x_norm = (x_ - x_min)/(x_max - x_min)
        pos_norm = np.stack((x_norm,y_norm),axis=-1)

        # select boundary points
        if chunks<2*k:
            bound_idx = np.arange(chunks) # select all points as boundary points
        else:
            bound_ratio = 0.1
            while True:
                h_y, h_x = bound_ratio, bound_ratio
                mask_boundary = (y_norm > 1-h_y) | (y_norm < h_y)|(x_norm > 1-h_x)|(x_norm < h_x)
                bound_idx = np.nonzero(mask_boundary)[0]
                bound_ratio = bound_ratio + 0.1
                if len(bound_idx)>=k: break

        # esimate distance of kth NN of the boundary points
        tree = KDTree(pos_norm)
        dd = tree.query(pos_norm[bound_idx],k=[k,],workers=-1)[0]
        max_dd = np.max(dd)

        # decide halo size and query halo points
        y_halo_size, x_halo_size = max_dd*(y_max-y_min), max_dd*(x_max-x_min)
        halo_bounds = [x_min-x_halo_size, y_min-y_halo_size, x_max+x_halo_size, y_max+y_halo_size]
        halo_idx = rtree.bbox_query(halo_bounds, x, y)
        #print(halo_idx.shape)
        y_halo, x_halo = y[halo_idx], x[halo_idx]

        # find input points: union of knn in halo points for points in the chunk 
        tree = KDTree(np.stack((x_halo, y_halo),axis=-1))
        in_idx = tree.query(np.stack((x_, y_),axis=-1),k=k,workers=-1)[1]
        in_idx = np.unique(in_idx)
        in_idx = halo_idx[in_idx] # idx in all points
        in_indices.append(in_idx)
        # assert np.all(np.isin(np.arange(start,end),in_idx))

        # indices that map points in the chunk from input points
        map_idx = (in_idx>=start) & (in_idx<end)
        map_idx = np.flatnonzero(map_idx)
        map_indices.append(map_idx)

    return in_indices, out_slices, map_indices

In [ ]:
#| hide
# in_indices_, out_slices_, map_indices_ = chunkwise_knn_mapping_(lat, lon, 40000)
# for i in range(len(in_indices)):
#     np.testing.assert_array_equal(in_indices[i], in_indices_[i])
#     np.testing.assert_array_equal(map_indices[i], map_indices_[i])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()